![puma logo](https://github.com/nasa/puma/raw/main/doc/source/puma_logo.png)
# Welcome to the PuMA tutorial

The objective of this notebook is to familiarize new users with the main datastructures that stand at the basis of the PuMA project, and outline the functions to compute material properties (please refer to these papers ([1](https://www.sciencedirect.com/science/article/pii/S2352711018300281), [2](https://www.sciencedirect.com/science/article/pii/S235271102100090X)) for more details on the software).

# Installation setup and imports

The first code block will execute the necessary installation and package import. 

If you are running this jupyter notebook locally on your machine, assuming you have already installed the software, then the installation step will be skipped.


In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install 'git+https://github.com/nasa/puma'
    !pip install -q piglet pyvirtualdisplay
    !apt-get -qq install xvfb

import numpy as np
import pumapy as puma
import pyvista as pv
import scipy.ndimage as nd
import os
import sys

if 'google.colab' in str(get_ipython()):
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(600, 400))
    display.start()  # necessary for pyvista interactive plots
    
else:  # NORMAL JUPYTER NOTEBOOK
    # for interactive slicer (only static allowed on Colab)
    %matplotlib widget

## Tutorial: Thermal Conductivity
In this tutorial we demonstrate how to compute the effective thermal conductivity of a material based on its microstructure and constituent properties. In this example, we compute the thermal conductivity of FiberForm, a carbon fiber based material. 

Note: The sample size used in this example is very small, well below the size needed for a representative volume of the sample. 

### Isotropic conductivity

We will show a thermal conductivity simulation based on a non-segmented representation of the material. In the example material used, the void phase is contained in grayscale range [0,89] and the solid phase is contained in the grayscale range of [90,255]. This range varies for each tomography sample.

The outputs of the thermal conductivity simulation are the effective thermal conductivity tensor, the steady state temperature profile, and the steady state heat flux. 

In [2]:
# Import an example tomography file of size 200^3 and voxel length 1.3e-6
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)

# Cropping the 200x200x200 image into a 100x100x100 sample
ws_fiberform.matrix = ws_fiberform.matrix[50:150, 50:150, 50:150]

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_material((0, 89), 0.0257)
# Next we set the conductivity of the solid phase to be 12 (typical value for carbon fiber)
cond_map.add_material((90, 255), 12)

# The thermal conductivity calculation needs to be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's', tolerance=1e-3, solver_type='cg')
k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's', tolerance=1e-3, solver_type='cg')
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's', tolerance=1e-3, solver_type='cg')

print("Effective thermal conductivity tensor:")
print("[", k_eff_x[0], k_eff_y[0], k_eff_z[0], "]")
print("[", k_eff_x[1], k_eff_y[1], k_eff_z[1], "]")
print("[", k_eff_x[2], k_eff_y[2], k_eff_z[2], "]")

Importing /Users/josephferguson/miniconda/envs/puma/lib/python3.9/site-packages/pumapy/data/200_fiberform.tif ... Done
Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to assemble matrices:  0.4958141169999992
Solving Ax=b using cg solver
Iteration: 1301, driving modified residual = 0.0009861698 --> target = 0.0010000000 ... Done
Time to solve:  32.606636559
Computing flux from converged temperature field ... Done

Effective tensor (first column): 
[0.8749046981927542, -0.21391987120312525, -0.11036496798052424]

Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to assemble matrices:  0.3579874780000054
Solving Ax

In [3]:
# Visualize both the temperature field and the fibers
p = pv.Plotter(notebook=True)
p.add_text("Fibers with Temperature field for x simulation")
puma.render_contour(ws_fiberform, cutoff=(90, 255), notebook=True, add_to_plot=p, plot_directly=False)
puma.render_volume(T_x[:, :2*T_x.shape[1]//3], solid_color=None, cmap='jet', notebook=True, add_to_plot=p, plot_directly=False)
p.show()

2022-07-02 14:48:13.381 (  75.458s) [          2270EA]       vtkThreshold.cxx:96    WARN| vtkThreshold::ThresholdBetween was deprecated for VTK 9.1 and will be removed in a future version.


ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [4]:
# Visualizing the flux magnitude: 
puma.render_volume(np.linalg.norm(q_x, axis=3), notebook=True, cmap='jet')

2022-07-02 14:48:14.617 (  76.694s) [          2270EA]       vtkThreshold.cxx:96    WARN| vtkThreshold::ThresholdBetween was deprecated for VTK 9.1 and will be removed in a future version.


ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

Below is an example of a thermal conductivity simulation, but now performed on a segmented image in order to show an alternative option.

In [5]:
# Segments the image. All values >= 90 are set to 1, and all values <90 are set to 0.
ws_fiberform.binarize(90)

# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.IsotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_material((0, 0), 0.0257)
# Next we set the conductivity of the solid phase to be 12 (typical value for carbon fiber)
cond_map.add_material((1, 1), 12)
# Note that the grayscale ranges have changed relative to the last example, since this image is already segmented

# The thermal conductivity calculation needs to be run for each of the three simulation directions. 
# For each simulation, a temperature gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance (optional) - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter (optional) - maximum number of iterations, defaults to 10,000
#.  7. solver_type (optional) - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's', tolerance=1e-3, solver_type='cg')
k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's', tolerance=1e-3, solver_type='cg')
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's', tolerance=1e-3, solver_type='cg')

print("Effective thermal conductivity tensor:")
print("[", k_eff_x[0], k_eff_y[0], k_eff_z[0], "]")
print("[", k_eff_x[1], k_eff_y[1], k_eff_z[1], "]")
print("[", k_eff_x[2], k_eff_y[2], k_eff_z[2], "]")


Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to assemble matrices:  0.4871802360000004
Solving Ax=b using cg solver
Iteration: 1301, driving modified residual = 0.0009861698 --> target = 0.0010000000 ... Done
Time to solve:  30.816589261999994
Computing flux from converged temperature field ... Done

Effective tensor (first column): 
[0.8749046981927542, -0.21391987120312525, -0.11036496798052424]

Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to assemble matrices:  0.3482955040000064
Solving Ax=b using cg solver
Iteration: 689, driving modified residual = 0.0009856476 --> target = 0.0010000000 ... Done
Ti

In [6]:
# Visualizing the temperature field: 
puma.render_volume(T_y, solid_color=None, notebook=True, cmap='jet')

2022-07-02 14:49:27.972 ( 150.047s) [          2270EA]       vtkThreshold.cxx:96    WARN| vtkThreshold::ThresholdBetween was deprecated for VTK 9.1 and will be removed in a future version.


ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [7]:
# Visualizing the flux magnitude: 
puma.render_volume(np.linalg.norm(q_y, axis=3), notebook=True, cmap='jet')

2022-07-02 14:49:28.709 ( 150.785s) [          2270EA]       vtkThreshold.cxx:96    WARN| vtkThreshold::ThresholdBetween was deprecated for VTK 9.1 and will be removed in a future version.


ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

### Anisotropic conductivity
Next we show how to compute the conductivity if the constituent phases are anisotropic themselves. This solver is significantly slower because of the higher complexity of the numerical scheme used, namely the Multi-Point Flux Approximation (MPFA) (please refer to [this paper](https://www.sciencedirect.com/science/article/abs/pii/S092702562030447X) for more details on the anisotropic conductivity solver in PuMA). For this reason, we scale the domain by half in order to keep the runtime reasonable.

In [8]:
# Import an example tomography file of size 100^3 and voxel length 1.3e-6
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)
ws_fiberform.rescale(0.5, False)

# detect the fiber orientation using the Structure Tensor
puma.compute_orientation_st(ws_fiberform, cutoff=(90, 255), sigma=1.4, rho=0.7)

Importing /Users/josephferguson/miniconda/envs/puma/lib/python3.9/site-packages/pumapy/data/200_fiberform.tif ... Done
Rescaled workspace size: (100, 100, 100)
First gradient computation ... Done
Blurring of gradients ... Done
Computing eigenvalue analysis ... Done


True

In [9]:
# visualize the detected orientation field
puma.render_orientation(ws_fiberform, notebook=True)

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [10]:
# Generating a conductivity map. This stores the conductivity values for each phase of the material
cond_map = puma.AnisotropicConductivityMap()
# First, we set the conductivity of the void phase to be 0.0257 (air at STP)
cond_map.add_isotropic_material((0, 89), 0.0257)
# Next we set the conductivity of the fiber phase to be 12 along the fiber and 0.7 across it
cond_map.add_material_to_orient((90, 255), 12., 0.7)

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cond_map - the conductivity values for each material phase
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance (optional) - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter (optional) - maximum number of iterations, defaults to 10,000
#.  7. solver_type (optional) - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

# When an anisotropic conductivity is fed, the solver automatically uses the MPFA finite volume method
k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's')
k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's')
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's')

print("Effective thermal conductivity tensor:")
print("[", k_eff_x[0], k_eff_y[0], k_eff_z[0], "]")
print("[", k_eff_x[1], k_eff_y[1], k_eff_z[1], "]")
print("[", k_eff_x[2], k_eff_y[2], k_eff_z[2], "]")

Initializing and padding domains ... Done
Approximate memory requirement for simulation: 916.99 MB
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Assembling b vector ... Done
Time to assemble matrices:  21.50332982500001
Solving Ax=b using bicgstab solver
Iteration: 549, driving modified residual = 0.0009994285 --> target = 0.0010000000 ... Done
Time to solve:  51.737833144000035
Computing fluxes ... 100.0% Done

Effective conductivity tensor (first column): 
[0.21129216777721985, -0.01041553787810401, -0.03670904977155545]

Initializing and padding domains ... Done
Approximate memory requirement for simulation: 916.99 MB
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Assembling b vector ... Done
Time to assemble matrices:  21.795465729
Solving Ax=b using bicgstab solver
Iteration: 588, driving modified residual = 0.0009849699 --> target = 0.00100000000 ... Done
Time to solve:  75.43211734699997
Computing fluxes ... 100.

If the local phases are isotropic, the anisotropic solver can still be used (although it would not be convenient because it is slower). As proof that the two solvers are actually giving the same answer, we could run the following case, in which we compute the orientation and then set the same conductivity to both the conductivity components (i.e. along and across a fiber):

In [11]:
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)
ws_fiberform.rescale(0.5, segmented=False)

cond_map = puma.IsotropicConductivityMap()
cond_map.add_material((0, 89), 0.0257)
cond_map.add_material((90, 255), 12)

print("\nIsotropic solver")
k_eff_x, T_x, q_x = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's', tolerance=1e-3)
k_eff_y, T_y, q_y = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's', tolerance=1e-3)
k_eff_z, T_z, q_z = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's', tolerance=1e-3)

puma.compute_orientation_st(ws_fiberform, cutoff=(90, 255), sigma=1.4, rho=0.7)

cond_map = puma.AnisotropicConductivityMap()
cond_map.add_isotropic_material((0, 89), 0.0257)
cond_map.add_material_to_orient((90, 255), 12., 12)

print("\nAnisotropic solver")
k_eff_x_ani, T_x_ani, q_x_ani = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'x', 's', tolerance=1e-3)
k_eff_y_ani, T_y_ani, q_y_ani = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'y', 's', tolerance=1e-3)
k_eff_z_ani, T_z_ani, q_z_ani = puma.compute_thermal_conductivity(ws_fiberform, cond_map, 'z', 's', tolerance=1e-3)

print("\nEffective conductivity using isotropic solver")
print("[", k_eff_x[0], k_eff_y[0], k_eff_z[0], "]")
print("[", k_eff_x[1], k_eff_y[1], k_eff_z[1], "]")
print("[", k_eff_x[2], k_eff_y[2], k_eff_z[2], "]")

print("\nEffective conductivity using anisotropic solver")
print("[", k_eff_x_ani[0], k_eff_y_ani[0], k_eff_z_ani[0], "]")
print("[", k_eff_x_ani[1], k_eff_y_ani[1], k_eff_z_ani[1], "]")
print("[", k_eff_x_ani[2], k_eff_y_ani[2], k_eff_z_ani[2], "]")


Importing /Users/josephferguson/miniconda/envs/puma/lib/python3.9/site-packages/pumapy/data/200_fiberform.tif ... Done
Rescaled workspace size: (100, 100, 100)

Isotropic solver
Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to assemble matrices:  0.4923489759999029
Solving Ax=b using bicgstab solver
Iteration: 583, driving modified residual = 0.0008472163 --> target = 0.0010000000 ... Done
Time to solve:  36.92179269600001
Computing flux from converged temperature field ... Done

Effective tensor (first column): 
[0.6428980008299361, -0.06245961366793659, -0.1561057283970486]

Approximate memory requirement for simulation: 227.13 MB
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up precondition

As you can see, the tensors that have been estimated are very similar. The slight differences are coming from the high tolerance that was 